<a href="https://colab.research.google.com/github/Hatr94/Jets_Classification/blob/main/Pre_Tranied_ViT_5types2000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import vit_b_16  # Import a pre-trained ViT model
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary as model_summary
from tqdm.auto import tqdm

# Definir transformaciones para los datos de entrenamiento y validación
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # Redimensionar las imágenes a 224x224 píxeles
  transforms.ToTensor(),          # Convertir las imágenes a tensores
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


Mounted at /content/drive


In [2]:

# Definir el tamaño del batch
batch_size = 32

# Crear los conjuntos de datos
train_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/imagenes/entre', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/imagenes/test1', train=False, download=True, transform=transform)

# Split train dataset into train and validation sets 80% and 20%
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train Data: {len(train_dataset)}")
print(f"Validation Data: {len(val_dataset)}")
print(f"Test Data: {len(test_dataset)}")

# Crear los DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



100%|██████████| 170M/170M [00:13<00:00, 13.0MB/s]


Extracting /content/drive/MyDrive/imagenes/entre/cifar-10-python.tar.gz to /content/drive/MyDrive/imagenes/entre


100%|██████████| 170M/170M [00:19<00:00, 8.94MB/s]


Extracting /content/drive/MyDrive/imagenes/test1/cifar-10-python.tar.gz to /content/drive/MyDrive/imagenes/test1
Train Data: 40000
Validation Data: 10000
Test Data: 10000


In [3]:

# Definir hiperparámetros
num_epochs = 5
lr = 3e-5  # Learning rate
gamma = 0.7 # Factor de reducción de la tasa de aprendizaje

# Cargar el modelo preentrenado
model = vit_b_16(pretrained=True)
model.heads.head = nn.Linear(model.heads.head.in_features, 10)  # Ajustar para CIFAR-10 (10 clases)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 172MB/s]


In [4]:
# Definir dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model.to(device)
if device == 'cuda':
   print(torch.cuda.get_device_name(0))

# Definir función de pérdida, optimizador y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

Device: cuda


In [6]:
# Definir función de entrenamiento
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_loss = float('inf')
    model_file_name = 'best_model.pt'
    save_path ='/content/drive/MyDrive/modelo' # you have to create the file and put the  pathname

    # Crear el directorio si no existe
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

        # Guardar el modelo si es el mejor hasta ahora
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), os.path.join(save_path, model_file_name))
            print('New best model saved!')

    print("Entrenamiento completado")

# Entrenar el modelo
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/5, Train Loss: 0.16060586393140255, Validation Loss: 0.1267754861954064
New best model saved!
Epoch 2/5, Train Loss: 0.0418866695208475, Validation Loss: 0.09818052808112931
New best model saved!
Epoch 3/5, Train Loss: 0.01377060623751022, Validation Loss: 0.09541607687429052
New best model saved!
Epoch 4/5, Train Loss: 0.004824392477492802, Validation Loss: 0.07230527737646918
New best model saved!
Epoch 5/5, Train Loss: 0.0029767566883470865, Validation Loss: 0.083396283140984
Entrenamiento completado


In [7]:

def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test images: {accuracy}%')

In [8]:
# Load the best model
model_file_name = '/content/drive/MyDrive/modelo/best_model.pt'
best_model_state_dict = torch.load(model_file_name)
model.load_state_dict(best_model_state_dict)

<ipython-input-8-6c49a036a1b0>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model_state_dict = torch.load(model_file_name)


<All keys matched successfully>

In [9]:
# Evaluate the model
evaluate_model(model, test_loader)

Accuracy on test images: 97.84%


In [10]:
test_loss = 0.0
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calcular precisión
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
accuracy = correct / total * 100
print(f"Test Loss: {avg_test_loss}, Test Accuracy: {accuracy}%")

Test Loss: 0.08394333005340891, Test Accuracy: 97.84%
